# Grade Saver   - By Suganth Kumar Thangavel

/*Documentation*/

Problem: 
    We(Students) face difficulties in remembering Assignment deadlines and are also lazy enough(Atleast me) to check them in USF canvas site. So we end up submitting them late and lose grades.

About this program:
    This program is built to scrape USF canvas site and collect the Assignment details for the respective student and based on his reminder settings it will send an automatic notification to his mobile reminding about the assignment. Most of us check our mobile often so it will be good reminder for us.
    
    
Required software/details/information:

1.chrome driver should be present in the system. The driver w.r.t your Chrome can be downloaded below.
link: http://chromedriver.storage.googleapis.com/index.html

2.Student credentials for canvas site(Password can be stored in secure file and program can be pointed to access the data from file).

3.Please ensure your calander in USF canvas site is selected with the subjects for you want the assignments to be tracked

4.Twilio account creation(Account SID and Auth code) and the created twilio phone number-- Twilio is a sms gateway service which allows to send sms. You can create and use a trial account. Twilio account can be created at https://www.twilio.com/.



In [1]:
#Required Standard Libraries for scraping and text message api
from selenium import webdriver
from datetime import datetime,timedelta
## Sending text messages 
import twilio.rest
import time

# Accessing Chrome Browser through Selenium and opening link

In [2]:
#Choose the chromedriver which suits your chrome version
Chromedriver = r'C:/Users/sugan/Desktop/chromedriver.exe'
driver = webdriver.Chrome(Chromedriver)
url="https://webauth.usf.edu/login?service=https%3A%2F%2Fusflearn.instructure.com%2Flogin%2Fcas"
driver.get(url)

In [3]:
#Initializing required elements
dates = []
content = []
info=[]
final_dict={}

# Sending Login details to open canvas

In [4]:
username = driver.find_element_by_id("username")
password = driver.find_element_by_id("password")
#Opening the password file to get my password. I wont show my password :P
with open('Password.txt') as fp:
    secret=fp.readline().strip(" ")
username.send_keys("suganthkumar")
password.send_keys(secret)

# Opening Agenda page of calendar

In [5]:
submit_button = driver.find_element_by_id("btn-submit")
submit_button.click()
calender = driver.find_element_by_id('global_nav_calendar_link')
calender.click()
agenda = driver.find_element_by_id("agenda")
agenda.click()
time.sleep(5) #Delay is required as the page may take time to load

# Scraping the Assignment details

In [6]:
### Getting the Assignment dates
for day in driver.find_elements_by_class_name('agenda-day'):
    el = day.find_element_by_class_name('agenda-date')
    for text in el.find_elements_by_class_name('screenreader-only'):
        dates.append(text.text)
        
###Scraping the Assignement details        
for eventlist in driver.find_elements_by_class_name('agenda-event__list'):
    item_list =[]
    for item in eventlist.find_elements_by_class_name('agenda-event__item'): 
        info= []
        for screentext in item.find_elements_by_class_name('screenreader-only'):
            if(screentext.text not in['Open event menu']):
                info.append(screentext.text)
        item_list.append(info)
    content.append(item_list) 

#Check to match the content with dates
# if(len(dates)==len(content)):
#     print("First Step successful")
    
## Print the Scraped details 
print(content)

[[['Assignment, Due 11:59pm ,', 'M3 Assignment', 'Not Completed', 'Calendar ISM6930.006S18 Stats Prog for Bus Analytics']], [['Assignment, Due 11:59pm ,', 'M4 Assignment', 'Not Completed', 'Calendar ISM6930.006S18 Stats Prog for Bus Analytics'], ['Assignment, Due 11:59pm ,', 'Assignment 2 - Object relational mapping', 'Not Completed', 'Calendar ISM6225.901S18 Distributed Information System'], ['Assignment, Due 11:59pm ,', 'Project 1 - SportsStore', 'Not Completed', 'Calendar ISM6225.901S18 Distributed Information System']], [['Assignment, Due 11:59pm ,', 'Cassandra Java', 'Not Completed', 'Calendar ISM6930.001S18 Big Data & Electronic Commerce']], [['Assignment, Due 11:59pm ,', 'S3 Assignment', 'Not Completed', 'Calendar ISM6930.001S18 Big Data & Electronic Commerce']], [['Assignment, Due 11:59pm ,', 'M5 Assignment', 'Not Completed', 'Calendar ISM6930.006S18 Stats Prog for Bus Analytics']], [['Assignment, Due 11:59pm ,', 'Cloudera Map-Reduce Tutorial - Utopia Vote Count', 'Not Complete

# Dictionary to hold the Assignments based on date and time

In [7]:
#Creating a Dictionary to store the Assignment dates and details as a dictionary
for n in range(0,len(dates)):
    h=[]
    for a in content[n]:
        h.append(a)
        final_dict[dates[n]]=h
        
#Current date and the date before which it has to notify about the assignment
current_date = datetime.now()




#Creating a new dictionary by changing the key to timestamp
new_dict={}
for keys in final_dict:
    temp = keys+" "+str(current_date.year)
    new_key = datetime.strptime(temp,"%A, %B %d %Y")
    new_dict[new_key]=final_dict[keys]
    
    
##Printing the final dictionary showing the timestamps   

for k in new_dict.keys():
    print("Timestamp",k)
    print("---------")
    print(new_dict[k])
    print("")

Timestamp 2018-02-26 00:00:00
---------
[['Assignment, Due 11:59pm ,', 'M3 Assignment', 'Not Completed', 'Calendar ISM6930.006S18 Stats Prog for Bus Analytics']]

Timestamp 2018-03-05 00:00:00
---------
[['Assignment, Due 11:59pm ,', 'M4 Assignment', 'Not Completed', 'Calendar ISM6930.006S18 Stats Prog for Bus Analytics'], ['Assignment, Due 11:59pm ,', 'Assignment 2 - Object relational mapping', 'Not Completed', 'Calendar ISM6225.901S18 Distributed Information System'], ['Assignment, Due 11:59pm ,', 'Project 1 - SportsStore', 'Not Completed', 'Calendar ISM6225.901S18 Distributed Information System']]

Timestamp 2018-03-09 00:00:00
---------
[['Assignment, Due 11:59pm ,', 'Cassandra Java', 'Not Completed', 'Calendar ISM6930.001S18 Big Data & Electronic Commerce']]

Timestamp 2018-03-16 00:00:00
---------
[['Assignment, Due 11:59pm ,', 'S3 Assignment', 'Not Completed', 'Calendar ISM6930.001S18 Big Data & Electronic Commerce']]

Timestamp 2018-03-19 00:00:00
---------
[['Assignment, Due 1

# Filtering and concatenating the Assignments based on check date

In [8]:
## Addition to current_date
check_date = current_date+timedelta(days=4)

## Concatenating the messages into a variable
messages="" #Create a string
counter = 1
for keys in new_dict:
    if((keys.date()>=current_date.date())&(keys.date()<=check_date.date())):
        #print(keys)
        messages = messages+" "+str(counter)+"."+str(keys.ctime())+str(new_dict[keys])
        counter+=1   

In [9]:
messages

''

# Twilio Account Authorization to send text messages

In [10]:
## A twilio account has to be created at https://www.twilio.com and the AccountSID and Auth should be provided to send sms
AccountSID="AC7fbfc68a6df31ee22a5d56f3389fd199"
with open('Auth.txt') as fp:
    auth=fp.readline().strip(" ")
    
Auth = auth
## Twilio Account Number and the phonenumber to which the msg has to be sent
from_no ="+12673858643"
to_no = "+18136099735"

# Function which calls twilio api and sends the messages

In [11]:
#function to send text messages
def text_messages(message):
    client=twilio.rest.Client(AccountSID,Auth)
    msg_info = client.messages.create(body=message,from_=from_no,to=to_no,provide_feedback=True)
    return msg_info

if(len(messages)>0):
    msg_info = text_messages(message=messages)
else:
    print("No message to send")

No message to send
